In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import re   # Regular expressions

In [4]:
def cropImage(ivSource, ivDestination, ivType):    # Type refers to image with one or two boxes to split


    img = cv2.imread(ivSource)

    h,w = img.shape[:2]
    mask = np.zeros((h,w), np.uint8)

    # Transform to gray colorspace and invert Otsu threshold the image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #_, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


    ### Perform morphologyEx (erosion followed by dilation)
    #kernel = np.ones((2,2),np.uint8)
   # opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)   
    imgCanny = cv2.Canny(gray,100,100)
#    plt.imshow(imgCanny)


    # play with kernel settings to see impact on output files
    kernel = np.ones((5,5),np.uint8)
    #imgEroded = cv2.dilate(gray,kernel,iterations=1)
    imgEroded = cv2.dilate(imgCanny,kernel,iterations=1)
    #plt.imshow(imgEroded)# Search for contours, select the biggest and draw it on the mask
    #_, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)


  #  contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    #contours1, hierarchy1 = cv2.findContours(imgEroded, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contours1, _ = cv2.findContours(imgEroded, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
   
    tempContour = sorted(contours1, key=cv2.contourArea, reverse=True)

        
 #   temp1 = np.zeros((h,w), np.uint8)
    
    if ivType == 1:
        x,y,w,h = cv2.boundingRect(tempContour[1])
        final_image = img[y:y+h, x:x+w]
        cv2.imwrite(ivDestination, final_image )
    elif ivType == 2:
        
        for index, _ in enumerate(range(2)):
          x,y,w,h = cv2.boundingRect(tempContour[index +1])
          final_image = img[y:y+h, x:x+w]
          temp_destination = ivDestination[:-4] + "_" + str(index +1)  + ivDestination[-4:]
          cv2.imwrite(temp_destination, final_image )
          
    
 
            
 
    

In [6]:

## Wool images not being processed correctly - look into processing of those and how to split those images

one_box_files = ["wash"]

directory = '..\Data\Raw'
target_directory = '..\Data\Prepared'

# Get all filenames in the directory
filenames = os.listdir(directory)

pattern = r"\b(" + "|".join(one_box_files) + r")\b"

# Print the filenames
for filename in filenames:
    rel_file = os.path.join(directory, filename)
    target_file = os.path.join(target_directory, filename)

    # Check if the string contains any word from the array using regular expressions
    if re.search(pattern, rel_file, flags=re.IGNORECASE):
        print("The string contains a word from the array.")
        cropImage(rel_file, target_file,  1)
    else:
        cropImage(rel_file, target_file,  2)


The string contains a word from the array.
